In [ ]:
import csv
import os
import numpy as np
import cv2
import keras
import tensorflow as tf

DATASET_PATH = "../../data/udacity_sim_data/"
VAL_PATH = "../../data/track1/seq1/"
def load_dataset(file_path):
    '''
    Loads dataset in memory
    '''
    dataset = []
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            try:
                dataset.append({'center':line[0], 'left':line[1], 'right':line[2], 'steering':float(line[3]), 
                            'throttle':float(line[4]), 'brake':float(line[5]), 'speed':float(line[6])/15.0 - 1})
            except:
                continue
    return dataset

dataset = load_dataset(os.path.join(DATASET_PATH, "driving_log.csv"))
val_dataset = load_dataset(os.path.join(VAL_PATH, "driving_log.csv"))

print("Loaded {} samples from file {}".format(len(dataset),DATASET_PATH))
print("Loaded {} samples from file {}".format(len(val_dataset),VAL_PATH))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

sample_file = os.path.join(DATASET_PATH, dataset[0]['center'].strip())
INPUT_IMAGE_ROWS, INPUT_IMAGE_COLS, INPUT_IMAGE_CHANNELS = 160, 320, 3
rows, cols = 160, 320

seq_len = 5
BATCH_SIZE = 5

train_set = dataset
valid_set = val_dataset

print("train set has {} elements".format(len(train_set)))
print("valid set has {} elements".format(len(valid_set)))

def data_generator(path, dataset, seq_len):
    batch_seq_images = np.zeros((BATCH_SIZE, seq_len, 160, 320, 3))
    batch_seq_speed = np.zeros((BATCH_SIZE, seq_len, 1))
    
    while 1:
        for i in range(BATCH_SIZE):
            while 1:
                index = np.random.randint(len(dataset))
                if index + seq_len <= len(dataset):
                    seq_speed = []
                    seq_images = []
                    for j in range(index, index + seq_len):
                        seq_speed.append(dataset[j]["speed"])
                        img = cv2.imread(path + dataset[j]["center"].strip())
                        #img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:, :, 1]
                        #img = np.asarray(img).reshape(160, 320, 1)
                        seq_images.append(img)
                    seq_speed = np.array(seq_speed)
                    seq_images = np.array(seq_images)
                    break
                else:
                    continue
            batch_seq_images[i] = seq_images
            batch_seq_speed[i] = seq_speed.reshape(seq_len, 1)
            #batch_seq_steering_angles[i] = seq_steering_angles

        # for ru
        yield batch_seq_images, batch_seq_speed[:, -1, :]

In [ ]:
# model from scratch


from keras.models import Sequential, Model
from keras.layers.core import Lambda, Dense, Activation, Flatten, Dropout
#from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers import GRU, LSTM, Conv2D, ConvLSTM2D
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, RMSprop

print("\nBuilding and compiling the model ...")

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(TimeDistributed(Lambda(lambda x: (x / 127.5) - 1.0), input_shape=(None, INPUT_IMAGE_ROWS, INPUT_IMAGE_COLS, INPUT_IMAGE_CHANNELS)))
# Block - conv
model.add(TimeDistributed(Conv2D(8, (11, 11), padding='valid', strides=(2, 2), kernel_initializer='glorot_uniform', activation='relu', name='Conv1')))
# Block - conv
model.add(TimeDistributed(Conv2D(16, (5, 5), padding='valid', strides=(2, 2), kernel_initializer='glorot_uniform', activation='relu', name='Conv2')))
# Block - conv
model.add(TimeDistributed(Conv2D(16, (5, 5), padding='valid', strides=(2, 2), kernel_initializer='glorot_uniform', activation='relu', name='Conv3')))
# Block - conv

model.add(ConvLSTM2D(16, (5, 5), strides=(1, 1), activation='relu', recurrent_activation='relu', kernel_initializer='glorot_uniform',
         recurrent_initializer='glorot_uniform', padding='valid', return_sequences=False, name='convlstm'))
model.add(Conv2D(16, (3, 3), padding='valid', kernel_initializer='glorot_uniform', activation='relu', name='Conv4'))
model.add(Flatten(name='flat'))
model.add(Dropout(0.20))
model.add(Dense(256, name='fc'))
model.add(Dense(1, name='ouput'))
model.summary()

adam = Adam(lr=0.001)
#rmsprop = RMSprop()
model.compile(loss='mean_squared_error', optimizer=adam)

In [ ]:
# model pretrained
from keras.models import Sequential, Model
from keras.layers.core import Lambda, Dense, Activation, Flatten, Dropout
#from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers import GRU, LSTM, Conv2D, ConvLSTM2D
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, RMSprop

from keras.applications.vgg16 import VGG16

print("\nBuilding and compiling the model ...")

# import VGG
vgg = VGG16(include_top=False, weights='imagenet', input_tensor=None, 
            input_shape=(rows, cols, 3), classes=1000)
layer_dict = dict([(layer.name, layer) for layer in vgg.layers])

cnn_model = Model(inputs=vgg.input, outputs=vgg.get_layer("block5_pool").output)

# develop a combined model
rcnn_model = Sequential()
rcnn_model.add(TimeDistributed(cnn_model, input_shape=(None, 160, 320, 3)))

rcnn_model.add(ConvLSTM2D(256, (5, 10), strides=(1, 1), activation='relu', recurrent_activation='relu', kernel_initializer='glorot_uniform',
               recurrent_initializer='glorot_uniform', padding='valid', return_sequences=False, name='convlstm'))
rcnn_model.add(Flatten(name='flat'))
rcnn_model.add(Dense(1, name='output'))

for layer in cnn_model.layers:
    layer.trainable = False

rcnn_model.summary()

adam = Adam(lr=0.01)
#rmsprop = RMSprop()
rcnn_model.compile(loss='mean_squared_error', optimizer=adam)

In [ ]:
import keras
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import math

class LifeCycleCallBack(keras.callbacks.Callback):
    
    def on_epoch_begin(self, epoch, logs={}):
        pass
    
    def on_epoch_end(self, epoch, logs={}):
        pass

    def on_batch_begin(self, batch, logs={}):
        pass
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
    def on_train_begin(self, logs={}):
        print('Beginning training')
        self.losses = []
        
    def on_train_end(self, logs={}):
        print('Ending Training')


NUM_EPOCHS = 250
        
lifecycle_callback = LifeCycleCallBack()

tensorboard = keras.callbacks.TensorBoard(log_dir='../../data/logs/speed_cnn_rnn/', histogram_freq=1, 
                                          write_graph=True, write_images=True)

checkpoint = keras.callbacks.ModelCheckpoint("../../data/weights/speed_cnn_rnn/model.h5", monitor='val_loss', verbose=0, 
                                             save_best_only=True, save_weights_only=True, mode='auto', period=1)

train_generator = data_generator(DATASET_PATH, train_set, seq_len)
valid_generator = data_generator(VAL_PATH, valid_set, seq_len)

steps_per_epoch = math.ceil((len(train_set) - seq_len)/(seq_len*BATCH_SIZE))
validation_steps = math.ceil((len(valid_set) - seq_len)/(seq_len*BATCH_SIZE))

print("\nSaving Model...")

model_json = rcnn_model.to_json()
with open("../../data/weights/speed_cnn_rnn/model.json", "w") as json_file:
    json_file.write(model_json)

#model.save_weights("model.h5")
print("Model Saved.")


history = rcnn_model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              steps_per_epoch=steps_per_epoch,
                              validation_steps=validation_steps,
                              epochs=NUM_EPOCHS, verbose=1, 
                              callbacks=[lifecycle_callback, checkpoint])

print("\nTraining the model ended.")
